# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Read city_data.csv here 
city_data_df = pd.read_csv("WeatherPy_Output\city_data.csv")
city_data_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Campechuela,20.2342,-77.2789,88.81,63,86,3.89,CU,1661032210
1,1,Jamestown,42.0970,-79.2353,83.79,47,100,6.91,US,1661032211
2,2,East London,-33.0153,27.9116,55.15,65,86,16.46,ZA,1661032211
3,3,Davila,18.4726,120.5763,75.18,78,81,2.48,PH,1661032211
4,4,Hermanus,-34.4187,19.2345,46.80,69,39,6.91,ZA,1661032212
...,...,...,...,...,...,...,...,...,...,...
569,569,Ovsyanka,55.9575,92.5519,54.57,98,99,5.12,RU,1661032394
570,570,Riyadh,24.6877,46.7219,93.34,12,0,6.20,SA,1661032198
571,571,Korla,41.7597,86.1469,72.55,37,44,16.84,CN,1661032395
572,572,Laguna,38.4210,-121.4238,106.14,25,0,5.75,US,1661032395


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key= g_key)

In [4]:
city_loc = city_data_df[["Lat","Lng"]].astype(float)
Humidity =city_data_df["Humidity"].astype(float)

In [5]:
# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '500px',
    }
fig = gmaps.figure(layout=figure_layout)

#Create Heat Layer

heat_layer= gmaps.heatmap_layer(city_loc, weights=Humidity,dissipating= False,
                               max_intensity=max(city_data_df["Humidity"]),
                                 point_radius=2)

#Add Layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='500px', width='900px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Filtering Data condition by condition

#Condition 1. Max Temp greater than 70F
con_1_df = city_data_df.loc[city_data_df["Max Temp"] >=70, :]

#Condition 2. Max Temp less than 80F
con_2_df= con_1_df.loc[con_1_df["Max Temp"] <= 80, :]

#Condition 3. Cloudiness greater than 70%
con_3_df=con_2_df.loc[con_2_df["Cloudiness"] >= 70, :]

#Condition 4. Humidity less than 60%
con_4_df= con_3_df.loc[con_3_df["Humidity"] <= 60, :]




### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Storing filtered cities data into a variable named hotel_df, removing the extra index column
hotel_df = con_4_df.drop(columns=['Unnamed: 0'])

#Adding a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"]=""

In [8]:
#Setting Base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Using a iterrows and for loop to retrieve location (latitude and longitude)
for index, row in hotel_df.iterrows():
    Lat = row["Lat"]
    Lng = row["Lng"]
    location = f"{Lat},{Lng}"
    
    #Setting Params
    params = {"location":location,"keyword":"hotel","radius":5000,"key":g_key}
    
    
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    
    #Performing request on each of the above cities
    response = requests.get(base_url, params = params).json()
    try:
        #Storing Hotel Name into the hotel_df
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        
        
        
        
    except (KeyError, IndexError):
          print ("Missing field/result...skipping.")
        
        #putting an Nan value for cities where hotel wasnt found
          hotel_df.loc[index, "Hotel Name"]=np.nan 
     
    
    
    

Retrieving Results for Index 211: Kropotkin.
Retrieving Results for Index 229: Arzgir.
Missing field/result...skipping.
Retrieving Results for Index 289: Sadovoye.
Missing field/result...skipping.
Retrieving Results for Index 336: Marzuq.
Missing field/result...skipping.
Retrieving Results for Index 350: Thunder Bay.
Retrieving Results for Index 361: Envira.
Missing field/result...skipping.
Retrieving Results for Index 456: Svatove.
Retrieving Results for Index 526: Mandera.
Retrieving Results for Index 549: Jinchang.
Retrieving Results for Index 573: Whitehorse.


In [9]:
#Viewing the dataframe with hotel names present:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
211,Kropotkin,45.4375,40.5756,71.71,57,91,12.35,RU,1661032278,Otel' Vizit
229,Arzgir,45.3728,44.2206,77.29,39,100,24.54,RU,1661032283,NaN
289,Sadovoye,47.7764,44.5089,73.67,19,100,20.83,RU,1661032301,NaN
336,Marzuq,14.4000,46.4667,79.36,53,98,6.46,YE,1661032315,NaN
350,Thunder Bay,48.4001,-89.3168,75.11,50,75,5.75,CA,1661032319,"Holiday Inn Express & Suites Thunder Bay, an I..."
361,Envira,-7.3000,-70.2167,73.85,54,99,2.37,BR,1661032326,NaN
456,Svatove,49.4103,38.1503,77.00,36,100,17.07,UA,1661032357,Myslyvsʹka Vatra
526,Mandera,3.9366,41.8670,78.35,56,98,20.20,KE,1661032381,Granada hotel Mandera
549,Jinchang,38.4953,102.1739,73.24,48,78,4.07,CN,1661032388,Jinchang Hotel
573,Whitehorse,60.7161,-135.0538,70.11,42,75,12.66,CA,1661032395,Best Western Gold Rush Inn


In [10]:
#Dropping all rows from the filtered dataframe for which hotels were not located and viewing the dataframe
clean_hotel_df = hotel_df.dropna(axis=0, how='any')
clean_hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
211,Kropotkin,45.4375,40.5756,71.71,57,91,12.35,RU,1661032278,Otel' Vizit
350,Thunder Bay,48.4001,-89.3168,75.11,50,75,5.75,CA,1661032319,"Holiday Inn Express & Suites Thunder Bay, an I..."
456,Svatove,49.4103,38.1503,77.00,36,100,17.07,UA,1661032357,Myslyvsʹka Vatra
526,Mandera,3.9366,41.8670,78.35,56,98,20.20,KE,1661032381,Granada hotel Mandera
549,Jinchang,38.4953,102.1739,73.24,48,78,4.07,CN,1661032388,Jinchang Hotel
573,Whitehorse,60.7161,-135.0538,70.11,42,75,12.66,CA,1661032395,Best Western Gold Rush Inn


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker)

# Display figure
fig

Figure(layout=FigureLayout(height='500px', width='900px'))